# Case 2: Cargo Ship
The ship MS Leiden is traveling on a route involving five harbors in north-
ern Europe: Rotterdam, Hamburg, Kiel, Aarhus, and Copenhagen. You have
been asked to help the Captain with the loading and unloading plan (for the
destination harbors) that produces

- even and well-balanced solution
- a solution that is easy to unload (so that containers that are earlier un-loaded are not beneath containers that are later unloaded)
- solutions that can integrate as many containers as possible

The following details about the problem are provided:
1. The ship is now docked in Rotterdam. After visiting Rotterdam, the ship
will be traveling to Hamburg, Aarhus, and Copenhagen, in that order. The
containers destined for Rotterdam have been unloaded and the containers
destined for the remaining three harbors have to be loaded on the ship.
2. MS Leiden is a rather small container ship. It has eight bays, three tiers
and four rows. The layout of the ship is shown in the figure below.
3. Containers should be loaded such that a container that has to be unloaded
earlier should be placed in a higher position.
4. Each cell in the above figure is able to hold a forty-foot container. That
is, the ship has room for 8 × 3 × 4 = 96 forty-foot containers.
5. A forty-foot container can be placed on-top of another forty-foot container,
but not on top of an empty cell. We assume that only forty-foot containers
are loaded on the ship. Each container has a destination port.
6. Each container $i$ has a certain weight $w_i$. If a container is much heavier
than another $i$ container $j$, say $w_i − w_j > δ_w$, then it is not allowed to
place $w_i$ on top of $w_j$ .
7. The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [393]:
from desdeo_problem import variable_builder, ScalarObjective, ScalarConstraint, MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs import NSGAIII, PPGA

import numpy as np

# desdeo.probdefs.Problem.__abstractmethods__ = set()
# desdeo docs: https://desdeo.readthedocs.io/en/latest/

![title](cargo1.png)

![title](cargo2.png)

![title](cargo3.png)

## Code

Balance score can be calculated as:
The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [403]:
# Create DATA
# n = np.random.randint(10, 96)
n = 10
h = np.random.randint(1, 4, n)
w = np.random.randint(2000, 4000, n)
DATA = np.array(list(zip(h,w)))

HARBORS = np.unique(DATA[:,0])

### Helper functions

In [404]:
def remove_harbor_containers(configuration, destination):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
        destination: integer
            The destination harbor.
    Returns
    -------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    """
    tiers = range(configuration.shape[1])
    for tier in tiers:
        idx_container = np.argwhere(configuration[:, tier, :, 0] == destination)
        for idx in idx_container:
            if tier == 2:
                configuration[idx[0],tier,idx[1],:] = (0,0)
            # If the container is not on the top tier, move the container above it down
            else:
                configuration[idx[0],tier,idx[1],:] = configuration[idx[0],tier+1,idx[1],:]
                if tier == 1:
                    configuration[idx[0],tier+1,idx[1],:] = (0,0)
                else:
                    configuration[idx[0],tier+1,idx[1],:] = configuration[idx[0],tier+2,idx[1],:]
    return configuration

def positions_to_matrix(positions):

    configuration = np.zeros((8,3,4,2), dtype=int)

    for i, position in enumerate(positions):

        layer = int(position // 32)
        position = position % 32
        row = int(position // 4)
        column = int(position % 4)
        configuration[row, layer, column, :] = DATA[i]
        
    return configuration

### Objective 1: Stability

In [405]:
def sum_tiers(configuration):

    only_weights = configuration[:, :, :, 1]
    summed_configuration = np.sum(only_weights, axis=1)

    return summed_configuration

def get_longitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=1)
    positions = np.arange(1, 9)

    center = np.sum(weights * positions) / np.sum(weights)

    return center

def get_lattitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=0)
    positions = np.arange(1, 5)

    center = np.sum(weights * positions) / np.sum(weights)

    return center

def calculate_centers(configuration):

    summed_configuration = sum_tiers(configuration)

    longitudinal_center = get_longitudinal_center(summed_configuration)
    lattitudinal_center = get_lattitudinal_center(summed_configuration)

    return longitudinal_center, lattitudinal_center

def get_score(configuration):

    longitudinal_center, lattitudinal_center = calculate_centers(configuration)

    long_error, latt_error = abs(longitudinal_center - 4.5), abs(lattitudinal_center - 2.5)

    return long_error + latt_error

def calculate_total_stability(candidate_solutions):

    stability_scores = []

    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)

        stability_score = 0
        for i in HARBORS:

            stability_score += get_score(configuration)
            configuration = remove_harbor_containers(configuration, i)
        
        stability_scores.append(stability_score)
    # print('Stability scores:',stability_scores)
    return stability_scores

### Objective 2: Unloading time

In [406]:
def get_unloading_time(candidate_solutions):

    unloading_times = []
    
    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)
        only_harbors = configuration[:, :, :, 0]

        unloading_time = 0
        for i in range(8):
            for j in range(4):
                stack = only_harbors[i, :, j]
                stack = stack[stack != 0]
                if len(stack) > 1:
                    if tuple(stack) == (2,1,3):
                        return 4
                    for layer in range(len(stack)-1):
                        if stack[layer] == 1:
                            if stack[layer+1] != 1:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] != 1 and stack[layer+1] != 1:
                                    unloading_time += 2
                        elif stack[layer] == 2:
                            if stack[layer+1] == 3:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] == 3 and stack[layer+1] == 3:
                                    unloading_time += 2
        unloading_times.append(unloading_time)
    # print('Unloading times:',unloading_times)
    return unloading_times

In [407]:
def func(stack):
    unloading_time = 0
    if tuple(stack) == (2,1,3):
        return 4
    for layer in range(len(stack)-1):
        if stack[layer] == 1:
            if stack[layer+1] != 1:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] != 1 and stack[layer+1] != 1:
                    unloading_time += 2
        elif stack[layer] == 2:
            if stack[layer+1] == 3:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] == 3 and stack[layer+1] == 3:
                    unloading_time += 2
    return unloading_time

In [408]:
func([2, 1, 3])

4

In [409]:
# Constants
delta_weight = 2000

# Constraints
# 1. Containers can't be placed on the same position
# 2. Containers can not levitate
# 3. Containers can only weigh delta_weight kg more than the container below it
# 
# Objectives
# 1. Minimize the long_error + latt_error (maximize the stability of the ship)
# 2. Minimize the unloading time of the ship

# Create variables
var_names = ["c"+str(id) for id in range(len(DATA))]

initial_values = [pos for pos in range(len(DATA))]
lower_bounds = [0] * len(DATA)
upper_bounds = [95.9999] * len(DATA)

variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)

# Create objectives
objectives = [
    ScalarObjective(name="Stability", evaluator=calculate_total_stability, maximize=False),
    ScalarObjective(name="Unloading time", evaluator=get_unloading_time, maximize=False)
    ]

# Create constraints
def overlapping_eval(candidates, fitnesses):
    
    constraint_values = []
    for positions in candidates:
        positions = [int(pos) for pos in positions]
        
        if len(positions) != len(set(positions)):
            constraint_values.append(-1)
        else:
            constraint_values.append(1)

    return constraint_values
    
def levitating_eval(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]
        
        breach = False

        for pos in positions:

            if pos < 32:
                continue
            else:
                # pos_below = np.arange(pos-32, -1, -32)
                pos_below = pos - 32
                if not pos_below in positions:
                    breach = True
                    break
        
        if breach:
            constraint_values.append(-1)
        else:
            constraint_values.append(1)
            
    return constraint_values

def weight_eval(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]

        pos_weights = list(zip(positions, DATA[:,1]))

        breach = False

        for pos, weight in pos_weights:
            if pos < 32:
                continue
            else:
                pos_below = pos - 32
                try:
                    weight_below = pos_weights[positions.index(pos_below)][1]
                    if weight > weight_below + delta_weight:
                        breach = True
                        break
                except:
                        breach = True
                        break
        if breach:
            constraint_values.append(-1)
        else:
            constraint_values.append(1)

    return constraint_values


constraints = [
    ScalarConstraint(name="Overlapping", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=overlapping_eval),
    ScalarConstraint(name="Levitating", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=levitating_eval),
    ScalarConstraint(name="Weight", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=weight_eval)
    ]

problem = MOProblem(objectives, variables, constraints)

# Evolutionary Algorithm

# Create an algorithm
evolver = NSGAIII(problem, 
                  n_iterations=10,
                  n_gen_per_iter=100,
                  population_size=100)

# evolver = PPGA(problem, 
#                     n_iterations=10,
#                     n_gen_per_iter=100,
#                     population_size=100)

# Run the algorithm
iter = 0
while evolver.continue_evolution():
    print('Generation:',iter)
    print('Best stability:',np.min(evolver.population.fitness[:,0]))
    print('Best unloading time:',np.min(evolver.population.fitness[:,1]),'\n')
    evolver.iterate()
    iter += 1

Generation: 0
Best stability: 0.8205861904027723
Best unloading time: 0.0 

Generation: 1
Best stability: 0.25945587579151486
Best unloading time: 0.0 

Generation: 2
Best stability: 0.23224675400846806
Best unloading time: 0.0 

Generation: 3
Best stability: 0.17815463360346184
Best unloading time: 0.0 

Generation: 4
Best stability: 0.15341697144714495
Best unloading time: 0.0 

Generation: 5
Best stability: 0.14893184704888052
Best unloading time: 0.0 

Generation: 6
Best stability: 0.14893184704888052
Best unloading time: 0.0 

Generation: 7
Best stability: 0.13972978374887424
Best unloading time: 0.0 

Generation: 8
Best stability: 0.1380103670769004
Best unloading time: 0.0 

Generation: 9
Best stability: 0.13466259330284158
Best unloading time: 0.0 



In [411]:
print(evolver.population.constraint)
# print(evolver.population.fitness)

[[ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. -1.]
 [ 1. -1. 